In [1]:
import sys
sys.path.append('../../src/')
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
import collections as coll
from similarity_check.SimilarityCheck import *
from synthetic_data_generation.generator import *
from utils import *
# Import libraries
from privacy_check.privacy_check import PrivacyCheck
from sdv.sequential.par import PARSynthesizer
from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata

/opt/anaconda3/envs/CRP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets = load_csvs(folder_name='./data/')

In [3]:
df = datasets["energy_dataset"]
df = df[["time", "generation biomass", "generation fossil brown coal/lignite", "generation fossil gas"]]
df["id"] = df.index
df["time"] = df["time"].apply(lambda x: x[:-6])

In [4]:
df

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,id
0,2015-01-01 00:00:00,447.0,329.0,4844.0,0
1,2015-01-01 01:00:00,449.0,328.0,5196.0,1
2,2015-01-01 02:00:00,448.0,323.0,4857.0,2
3,2015-01-01 03:00:00,438.0,254.0,4314.0,3
4,2015-01-01 04:00:00,428.0,187.0,4130.0,4
...,...,...,...,...,...
35059,2018-12-31 19:00:00,297.0,0.0,7634.0,35059
35060,2018-12-31 20:00:00,296.0,0.0,7241.0,35060
35061,2018-12-31 21:00:00,292.0,0.0,7025.0,35061
35062,2018-12-31 22:00:00,293.0,0.0,6562.0,35062


In [5]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)
metadata.update_column(
    "id",
    sdtype='id',
)
metadata.set_sequence_key("id")
metadata.update_column(
    column_name='time',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S')
metadata.set_sequence_index(column_name='time')

In [6]:
metadata

{
    "columns": {
        "time": {
            "sdtype": "datetime",
            "datetime_format": "%Y-%m-%d %H:%M:%S"
        },
        "generation biomass": {
            "sdtype": "numerical"
        },
        "generation fossil brown coal/lignite": {
            "sdtype": "numerical"
        },
        "generation fossil gas": {
            "sdtype": "numerical"
        },
        "id": {
            "sdtype": "id"
        }
    },
    "sequence_key": "id",
    "sequence_index": "time",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Possible methods
For the generation of synthetic datasets with time series data, we have several methods at our disposal:
- PARSynthesizer (```sdv```): Probabilistic AutoRegressive model that allows learning multi-type, multivariate timeseries data and later on generate new synthetic data that has the same format and properties as the learned one.
- DGAN (```gretel-synthetics```)

In [7]:
# Step 1: Create the synthesizer
synthesizer = PARSynthesizer(metadata, verbose = True)

# Step 2: Train the synthesizer
synthesizer.fit(df)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

/opt/anaconda3/envs/CRP/lib/python3.10/site-packages/deepecho/models/par.py:125: RuntimeWarning: Mean of empty slice
  'mu': np.nanmean(x[i]),
/opt/anaconda3/envs/CRP/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
  0%|          | 0/128 [00:10<?, ?it/s]


KeyboardInterrupt: 